In [1]:
import sys 
sys.path.append('../modelling/')

In [2]:
from main_multitask_multimodal import LLM_MultitaskMultimodal

In [3]:
model = LLM_MultitaskMultimodal.load_from_checkpoint('../modelling/models/multitask_multimodal_multilingual/version_9/epoch=1-step=2600.ckpt/pytorch_model.bin')

INFO:root:Unused kwargs when getting google/mt5-base: {'distance_func': 'cosine', 'loss_type': 'cross-entropy', 'margin': None, 'hidden_states_type': 'encoder-last', 'add_simcse': False, 'manual_loss_type': 'manual_mse', 'auto_task_weight': False, 'multitask_specs_dict': {'clm_singlemodal_wishquery2tax': None}, 'head_dict': {}}
/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [4]:
model.transformer.save_pretrained('hf_models/query_classify_v2')

In [5]:
model.tokenizer.save_pretrained('hf_models/query_classify_v2')

('hf_models/query_classify_v2/tokenizer_config.json',
 'hf_models/query_classify_v2/special_tokens_map.json',
 'hf_models/query_classify_v2/spiece.model',
 'hf_models/query_classify_v2/added_tokens.json',
 'hf_models/query_classify_v2/tokenizer.json')

In [6]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("hf_models/query_classify_v2")
model_ort = ORTModelForSeq2SeqLM.from_pretrained("hf_models/query_classify_v2", from_transformers=True)

inputs = tokenizer("My name is Eustache and I like to", return_tensors="pt")

gen_tokens = model_ort.generate(**inputs)
outputs = tokenizer.batch_decode(gen_tokens)

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/modeling_utils.py:701: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if causal_mask.shape[1] < attention_mask.shape[1]:
In-place op on output of tensor.shape. See https://pytorch.org/docs/master/onnx.html#avoid-inplace-operations-when-using-tensor-shape-in-tracing-mode
In-place op on output of tensor.shape. See https://pytorch.org/docs/master/onnx.html#avoid-inplace-operations-when-using-tensor-shape-in-tracing-mode
/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be r

In [7]:
model_ort.save_pretrained("hf_models/query_classify_v2")